In [ ]:
## Generate the summary metrics using the generated dataset

In [ ]:
! pip install --user pandas pyarrow fastparquet

In [ ]:
! pip install --user imblearn

In [ ]:
! pip install pyyaml

In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('/repos/smote_msfb/helper_functions')
from summarize_dataset_with_hubness import summarize_dataset_with_hubness
from create_folder_if_not_exists import create_folder_if_not_exists
from run_single_experiment import run_single_experiment

In [2]:
folder_name = 'Burczynski'

In [3]:
path = "/repos/smote_msfb/public_datasets/"+ folder_name + "/"

In [4]:
output_dir = "/domino/datasets/local/smote_msfb/public_datasets/" + folder_name + "/"

In [5]:
## Create summary metric dataset for this public dataset
df = pd.read_csv(path + "final_data.csv.zip")
#df = pd.read_parquet( path + "processed_dataset.parquet" , engine="pyarrow")

In [6]:
df.head(3)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_22274,f_22275,f_22276,f_22277,f_22278,f_22279,f_22280,f_22281,f_22282,target
0,0,0,0,1,0,1,1,0,0,0,...,1,0,1,0,1,1,0,1,0,0
1,0,0,1,0,1,1,1,0,0,0,...,1,0,0,1,0,1,0,1,1,0
2,0,0,1,0,0,0,1,0,1,0,...,0,1,1,1,0,0,0,0,0,0


In [7]:
# Suppose df is your dataset already loaded into memory
# Features: f_*
feature_cols = [col for col in df.columns if col.startswith("f_")]

# Targets: target_*
target_cols = [col for col in df.columns if col.startswith("target")]

In [8]:
print("No. of target columns :", len(target_cols))

No. of target columns : 1


In [9]:
for target in target_cols: 
    X = df[feature_cols]
    y = df[[target]]  # keep as DataFrame
    combined_df = pd.concat([X, y], axis=1).reset_index(drop=True)
    
    # Rename target column to "target"
    combined_df = combined_df.rename(columns={target: "target"})    
    combined_df = combined_df.astype(int)
     
    counts = combined_df['target'].value_counts()
    print(counts)

    # Skip if minority class (<50 samples of 1)
    if counts.get(1, 0) < 10:
        print(f"Skipping {target} because it has fewer than 10 positives.")
        continue
    
    output_loc = output_dir + str(target) + "/"
    create_folder_if_not_exists(output_loc)
    
    # Redirect stdout to file
    log_file_path = output_loc + "output.txt"
    with open(log_file_path, "w") as f:
        # Temporarily redirect stdout
        old_stdout = sys.stdout
        sys.stdout = f
        try:
            print("Balance of target variable :",counts )
            run_single_experiment(output_loc, combined_df, mode="old")
        finally:
            # Restore stdout even if experiment fails
            sys.stdout = old_stdout
        

target
0    101
1     26
Name: count, dtype: int64
Folder '/domino/datasets/local/smote_msfb/public_datasets/Burczynski/target/' already exists.


/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
